In [19]:
# importando as bibliotecas
import re
import pandas as pd
import glob

taxas = []

# cria o molde do dicionario e transforma em dataframe
molde = {'Projeto': None,
         'Achou o termo "taxa interna de retorno"?': None,
         'Achou o termo "TIR"?': None,
         'Valor da TIR': None}
molde = pd.DataFrame(molde, index=[1])


# coloque aqui o caminho da pasta com os txts
path = "H:\\OneDrive\\FGV\\6 semestre\\GVCODE Projetos\\PDFs Grivol\\PDFs\\*.txt"

# coloque aqui o caminho da pasta que você quer que o excel final apareça
output = "H:\\OneDrive\\FGV\\6 semestre\\GVCODE Projetos\\PDFs Grivol\\TIR dos Projetos400.xlsx"

# faz uma lista de arquivos
projetos = glob.glob(path)


# loop macro
for projeto in projetos:

    # abre o projeto e copia o seu conteúdo
    f = open(projeto, 'r', encoding="utf8")  # abre o arquivo
    content = f.read()  # copia seu conteúdo
    f.close()  # fecha o arquivo

    # cria o um dicionário de resultado vazio
    resultado = {}

    ### Vamos agora preencher o dicionário

    # pega o nome do projeto e salva no dicionario
    nome = re.search(r'projeto\d*', projeto).group()
    resultado['Projeto'] = nome

    # procura o termo "taxa interna de retorno"
    achou_termo_taxa_interna_de_retorno = re.findall(
        r'taxa interna de retorno', content, re.IGNORECASE)

    # salva quantas vezes no dicionario
    if len(achou_termo_taxa_interna_de_retorno) == 0:
        resultado['Achou o termo "taxa interna de retorno"?'] = 'Não'
    elif len(achou_termo_taxa_interna_de_retorno) == 1:
        resultado['Achou o termo "taxa interna de retorno"?'] = '1 vez'
    else:
        resultado['Achou o termo "taxa interna de retorno"?'] = f'{len(achou_termo_taxa_interna_de_retorno)} vezes'

    # procura o termo "TIR"
    achou_termo_TIR = re.findall(r'\sTIR\s', content)

    # salva quantas vezes no dicionario
    if len(achou_termo_TIR) == 0:
        resultado['Achou o termo "TIR"?'] = 'Não'
    elif len(achou_termo_TIR) == 1:
        resultado['Achou o termo "TIR"?'] = '1 vez'
    else:
        resultado['Achou o termo "TIR"?'] = f'{len(achou_termo_TIR)} vezes'

    # Aqui vamos procurar a taxa interna de retorno em si

    # salva os 300 caractéres a frente dos termos "taxa interna de retorno" e "TIR"
    correspondencia_TIR = re.findall(r'TIR[\s\S]{400}', content)
    correspondencia_taxa = re.findall(
        r'taxa interna de retorno[\s\S]{300}', content, re.IGNORECASE)

    # cria o output das taxas
    taxa_TIR = []  # relativo as correspondencias ao termo "TIR"
    taxa_tir = []  # relativo as correspondencias ao termo "taxa interna de retorno"

    # loop encontra a taxa da amostra de 300 caractéres
    # se a taxa não estiver presente nos seguintes 300 caractéres, o loop retorna vazio
    # o número 300 foi escolhido por tentativa e erro
    # se o número de caractéres for grande demais, o algoritmo pode retornar um número que não é a tir
    # se o número de caractéres for pequeno demais, o algoritmo pode não encontrar a TIR
    # não existe um número mágico, é tudo uma questão de escolha entre minimizar falso positivo ou falso negativo
    for correspondencia in correspondencia_TIR:
        taxa_TIR.append(re.findall(r'\d+,\d+%|\d+\.\d+%|\d+%',correspondencia))  # inclui número decimal
    for correspondencia in correspondencia_taxa:
        taxa_tir.append(re.findall(r'\d+,\d+%|\d+\.\d+%|\d+%',correspondencia))  # inclui número decimal

    # agora tem que salvar os resultados no dicionário
    taxas = [taxa_tir, taxa_TIR]  # junta as taxas
    resultado['Valor da TIR'] = taxas

    # concatena com o DataFrame
    resultado = pd.DataFrame.from_dict(resultado, orient='index').T
    molde = pd.concat([molde, resultado])


# fazendo as últimas mudanças no dataframe
molde.reset_index(inplace=True)
molde.drop(0, inplace=True)
molde.drop('index', axis=1, inplace=True)

# salvando o dataframe em um excel
molde.to_excel(excel_writer=output)